# Proximal Policy Optimization

В практической реализации существует два варианта реализации алгоритма PPO:
* выполняет обновление, ограниченное KL, как TRPO, но штрафует KL-расхождение в целевой функции вместо того, чтобы делать его жестким ограничением, и автоматически регулирует коэффициент штрафа в процессе обучения, чтобы он масштабировался соответствующим образом.
* не содержит в целевой функции члена KL-дивергенции и вообще не имеет ограничения. Вместо этого полагается на специализированный клиппинг

<img src="https://spinningup.openai.com/en/latest/_images/math/e62a8971472597f4b014c2da064f636ffe365ba3.svg">

Спойлер: клиппинг - не самое главное в PPO, как это могло показаться на первый взгляд. Алгоритм PPO работает во многом и за счет небольших дополнительных улучшений. Подробнее: https://arxiv.org/pdf/2005.12729.pdf

# Устанавливаем зависимости

In [ ]:
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !apt -qq update -y
    !apt -qq install swig -y
    !pip -q install box2d-py
    !pip -q install "gymnasium[classic-control, box2d, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

45 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,116 kB of archives.
After this operation, 5,542 kB of additional disk space will be used.
Selecting previously unselected package swig4.0.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubuntu1_all.deb ...
Unpacking swig (4.0.2-1ubuntu1) ...
Setting up swig4.0 (4.0.2-1ubuntu1) ...
Setting up swig (4.0.2-1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 5.0 MB/s eta 0:00:00

In [ ]:
import torch
import torch.nn as nn
from torch.distributions import Categorical
import gymnasium as gym
import numpy as np
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Память

In [ ]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.is_terminals = []

    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.is_terminals[:]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Сеть Actor-Critic

In [ ]:
class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, hidden):
        super(ActorCritic, self).__init__()

        # actor: 2 hidden + output
        # self.action_layer = nn.Sequential(..., nn.Softmax(dim=-1))
        # 3 linear layers
        self.action_layer = nn.Sequential(
            nn.Linear(state_dim, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden),
            nn.Tanh(),
            nn.Linear(hidden, action_dim),
            nn.Softmax(dim=-1)
        )

        # critic: 2 hidden + output
        # self.value_layer = nn.Sequential(...)
        # 3 linear layers
        self.value_layer = nn.Sequential(
            nn.Linear(state_dim, hidden),
            nn.Tanh(),
            nn.Linear(hidden, hidden),
            nn.Tanh(),
            nn.Linear(hidden, 1)
        )

    def forward(self):
        raise NotImplementedError

    def act(self, state, memory):
        state = torch.from_numpy(state).float().to(device)

        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)
        action = dist.sample()

        # сохраняем в память: state, action, log_prob(action)
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(dist.log_prob(action))

        return action.item()

    def evaluate(self, state, action):
        action_probs = self.action_layer(state)
        dist = Categorical(action_probs)

        action_logprobs = dist.log_prob(action)
        dist_entropy = dist.entropy()

        state_value = self.value_layer(state)
        return action_logprobs, torch.squeeze(state_value), dist_entropy


# PPO policy

In [ ]:
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs

        self.policy = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(), lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())

        self.MseLoss = nn.MSELoss()

    def update(self, memory):
        # Monte Carlo оценка вознаграждений:
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(memory.rewards), reversed(memory.is_terminals)):
            # обнуляем накопленную награду, если попали в терминальное состояние
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + self.gamma * discounted_reward
            rewards.insert(0, discounted_reward)

        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        # выполните нормализацию вознаграждений (r - mean(r)) / std(r + 1e-5):
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)

        # конвертация list в tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.stack(memory.logprobs).to(device).detach()

        # оптимизация K epochs:
        for _ in range(self.K_epochs):
            # получаем logprobs, state_values, dist_entropy от стратегии:
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)

            # находим отношение стратегий (pi_theta / pi_theta__old), через logprobs и old_logprobs.detach():
            ratios = torch.exp(logprobs) / torch.exp(old_logprobs)

            # считаем advantages
            advantages = rewards - state_values

            # Находим surrogate loss:
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5 * self.MseLoss(state_values, rewards) - 0.01 * dist_entropy

            # делаем шаг градиента
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

        # копируем веса
        self.policy_old.load_state_dict(self.policy.state_dict())

# Основной цикл

In [ ]:
# env_name = "CartPole-v1"
env_name = "LunarLander-v2"
# env_name = "MountainCar-v0"

env = gym.make(env_name)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
render = False
solved_reward = 200  # останавливаемся если avg_reward > solved_reward
log_interval = 20  # печатаем avg reward  в интервале
max_episodes = 50000  # количество эпизодов обучения
max_timesteps = 500  # максимальное кол-во шагов в эпизоде
n_latent_var = 64  # кол-во переменных в скрытых слоях
update_timestep = 2000  # обновляем policy каждые n шагов
lr = 0.001 # learning rate
betas = (0.9, 0.999) # betas для adam optimizer
gamma = 0.99  # discount factor
K_epochs = 4  # количество эпох обноеления policy
eps_clip = 0.1  # clip параметр для PPO
random_seed = None

if random_seed:
    torch.manual_seed(random_seed)
    env.seed(random_seed)

memory = Memory()
ppo = PPO(state_dim, action_dim, n_latent_var, lr, betas, gamma, K_epochs, eps_clip)
print(lr, betas)

# переменные для логирования
running_reward = 0
avg_length = 0
timestep = 0

# цикл обучения
for i_episode in range(1, max_episodes + 1):
    state, _ = env.reset()
    for t in range(max_timesteps):
        timestep += 1

        # используем policy_old для выбора действия
        action = ppo.policy_old.act(state, memory)
        state, reward, terminated, truncated, _ = env.step(action)

        # сохраняем награды и флаги терминальных состояний:
        memory.rewards.append(reward)
        memory.is_terminals.append(terminated)

        # выполняем обновление
        if timestep % update_timestep == 0:
            ppo.update(memory)
            memory.clear_memory()
            timestep = 0

        running_reward += reward
        if render:
            env.render()
        if terminated or truncated:
            break

    avg_length += t

    # останавливаемся, если avg_reward > solved_reward
    if running_reward > (log_interval * solved_reward):
        print("########## Принято! ##########")
        torch.save(ppo.policy.state_dict(), './PPO_{}.pth'.format(env_name))
        break

    # логирование
    if i_episode % log_interval == 0:
        avg_length = int(avg_length / log_interval)
        running_reward = int((running_reward / log_interval))

        print('Episode {} \t avg length: {} \t reward: {}'.format(i_episode, avg_length, running_reward))
        running_reward = 0
        avg_length = 0

0.001 (0.9, 0.999)
Episode 20 	 avg length: 92 	 reward: -223
Episode 40 	 avg length: 91 	 reward: -180
Episode 60 	 avg length: 93 	 reward: -192
Episode 80 	 avg length: 89 	 reward: -150
Episode 100 	 avg length: 97 	 reward: -138
Episode 120 	 avg length: 106 	 reward: -181
Episode 140 	 avg length: 105 	 reward: -156
Episode 160 	 avg length: 114 	 reward: -178
Episode 180 	 avg length: 105 	 reward: -145
Episode 200 	 avg length: 105 	 reward: -122
Episode 220 	 avg length: 96 	 reward: -119
Episode 240 	 avg length: 99 	 reward: -118
Episode 260 	 avg length: 101 	 reward: -116
Episode 280 	 avg length: 120 	 reward: -130
Episode 300 	 avg length: 114 	 reward: -128
Episode 320 	 avg length: 125 	 reward: -112
Episode 340 	 avg length: 118 	 reward: -123
Episode 360 	 avg length: 137 	 reward: -100
Episode 380 	 avg length: 119 	 reward: -90
Episode 400 	 avg length: 116 	 reward: -94
Episode 420 	 avg length: 115 	 reward: -88
Episode 440 	 avg length: 118 	 reward: -61
Episod

### Дополнительное задание: Попробуйте обучить алгоритм PPO, используя более сложную среду (например LunarLander-v2), для этого вам потребуется подобрать некоторые гиперпараметры.

**Работа со средой LunarLander в colab - установка pybox2d:**
1. Устанавливаем пакеты:
```bash
!apt-get install swig -y
!pip install gymnasium[box2d]
```
2. Перезапускаем runtime.